# Studio Projektowe

## Pobranie danych z API NBP

In [2]:
import requests
import pandas as pd
import numpy as np
import datetime

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split


def loadData():
    col_names = ['date', 'price']
    tmp_data = []
    for i in range(2013, 2020):
        tmp_df = pd.DataFrame(columns=col_names)
        response = requests.get(f'http://api.nbp.pl/api/cenyzlota/{i}-01-01/{i}-12-31?format=json')
        response = response.json()
        response = pd.DataFrame(response)
        tmp_df['date'] = response['data']
        tmp_df['price'] = response['cena']
        tmp_data.append(tmp_df)
    data = pd.concat(tmp_data, ignore_index=True)
    
    today = datetime.date.today()
    tmp_df = pd.DataFrame(columns=col_names)
    response = requests.get(f'http://api.nbp.pl/api/cenyzlota/2020-01-01/{today}?format=json')
    response = response.json()
    response = pd.DataFrame(response)
    tmp_df['date'] = response['data']
    tmp_df['price'] = response['cena']
    data = pd.concat([data, tmp_df], ignore_index=True)
    return data

def saveDataToFile(data):
    data.to_csv('data.csv', index=False)

data = loadData()
saveDataToFile(data)

## Przetwarzanie danych

In [3]:
def addTimeColumns(data):
    data['date'] = data['date'].astype('datetime64')
    data['year'] = data['date'].dt.year
    data['month'] = data['date'].dt.month
    data['day'] = data['date'].dt.day
    data['weekday'] = data['date'].dt.weekday
    return data

data = addTimeColumns(data)

years = [data[data.year == i] for i in range(2013, 2021)]

months = [[years[i][data.month == j] for j in range(1,13)] for i in range(8)]


C:\Users\test\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]
C:\Users\test\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]
C:\Users\test\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]
C:\Users\test\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]
C:\Users\test\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]
C:\Users\test\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]
C:\Users\test\Anaconda3\lib\

## Podział danych na zbiory treningowe i testowe

In [18]:
data.index = data['date']
df = data[['price']]
print(df)

# a variable for predicting 30 days out into the future
forecast_out = 69

df['Prediction'] = df[['price']].shift(-forecast_out)
data.tail(40)

# Convert the dataframe to a numpy array
X = np.array(df.drop(['Prediction'],1))

# Remove the last '30' rows
X = X[:-forecast_out]

# Convert the dataframe to a numpy array 
y = np.array(df['Prediction'])

# Get all of the y values except the last '30' rows
y = y[:-forecast_out]

# Split the data into training set and testing set
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.015, shuffle=False)

x_forecast = np.array(df.drop(['Prediction'], 1))[-forecast_out:]

             price
date              
2013-01-02  165.83
2013-01-03  166.97
2013-01-04  167.43
2013-01-07  167.98
2013-01-08  167.26
...            ...
2020-12-01  211.74
2020-12-02  217.55
2020-12-03  217.04
2020-12-04  217.86
2020-12-07  217.83

[2000 rows x 1 columns]


C:\Users\test\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
x_train

array([[165.83],
       [166.97],
       [167.43],
       ...,
       [229.12],
       [228.16],
       [227.66]])

In [20]:
# Support Vector Machine (Regression)
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_rbf.fit(x_train, y_train)

svm_confidence = svr_rbf.score(x_test, y_test)
print("SVM Confidence: ", svm_confidence) # the best possible score is 1.0

svm_prediction = svr_rbf.predict(x_forecast)

# SVM Prediction output
print("SVM Prediction Output")
print(svm_prediction)

# Linear Regression
lr = LinearRegression()
lr.fit(x_train, y_train)

lr_confidence = lr.score(x_test, y_test)
print("LR Confidence: ", lr_confidence)

lr_prediction = lr.predict(x_forecast)

# Linear Regression Prediction Output
print("Linear Regression Prediction Output")
print(lr_prediction)

SVM Confidence:  -4.53815246916549
SVM Prediction Output
[229.35032904 228.94248544 228.94393177 229.00454046 229.21307208
 230.14785925 232.69520464 217.59634476 208.19094496 233.90292088
 227.0833619  233.67441563 222.59534375 233.80167757 231.58813684
 229.10391714 233.86808954 232.88698729 229.19777604 229.49670668
 232.63575397 210.45056202 233.41755999 232.73459174 233.80167757
 233.89367218 233.89833257 233.88641508 231.46933539 233.11509135
 233.85256021 235.01320345 233.6657457  222.95895873 196.89884505
 211.93181574 210.45056202 190.80937914 221.85735763 223.19926804
 223.7923463  203.97646142 226.36056271 211.25718558 191.39323505
 185.80041498 189.14416083 202.40008266 194.16213877 201.22563904
 235.88159433 236.67615628 236.78452262 233.59886443 236.78452262
 236.28914571 235.95625452 234.72997544 236.25861847 233.74330524
 232.02046806 231.18110046 231.12291726 233.99416322 232.19424664
 232.4863186  234.54879451 231.62814094 231.69305317]
LR Confidence:  -3.070294692511